![title](image.png)

<h1>Feather Vision</h1>


<style>
	p {
		line-height: 180%;
	}
</style>
<h2>1. Project description </h2>

<p>Ornithology enthusiasts and zoologists are people we thank for protecting and discovering new species on almost daily basis. Their jobs are alredy extremely difficult with the amount of information they have to keep track of while always being on the lookout for rare sightings of exotic birds.
With FeatherVision their job will be made much easier, when it comes to identifying numerous different species of birds.</p>
<p>The objective of our project is to create a machine learning model capable of accurately classifying bird species based on visual attributes in images. This way anyone could become a beginner ornithologist by simply capturing an image of their own and find out the details of the fauna in their local area. And for those advanced in the field, they could spend more time taking care of endangered species and researching by decreasing the precious time needed to properly identify these species.</p> 
<p> The world of birds is enormous and with FeatherVision you will never get lost in it again!  </p>

<h2>Framing the problem</h2>

<p>Our goal is to be able to identify bird species by just looking at them. You simply take a picture of the bird and our trained model will tell you what kind of bird this is. Since our dataset is fully lableled, meaning all the pictures have a label (bird species name) that the model can check, we chose the <b>Supervised Learning</b> approach. This means we will most likely choose a classification model to classify an image to a specie's name.</p>

<style>
	p {
		line-height: 180%;
	}
</style>
<h2>2. About the data</h2>
<p>
We have decided to use birds classification dataset from <a href="https://www.kaggle.com/datasets/gpiosenka/100-bird-species">kaggle</a>. It has a large number of training images (almost 85,000!) and includes over 500 different kinds of birds. This will help our model learn to identify lots of different bird species. We wanted a dataset with lots of high-quality images, good bird visibility, and this one fits us well. Important note is that all the images are photographed and not AI generated. While there are a few things to keep in mind, like each picture only having one bird, which can be good or bad, and also the images are 80% male, we believe this dataset is a good choice for our bird classification project.
</p>

<h3>Size and type of data</h3>
<p>
As mentioned earlier, the dataset contains almost 85K good quality images.   
<ul>
	<li>The images are sized 224 X 224 and are in JPG format.</li>
	<li>The average image size is around 20kB.</li>
	<li>The images are photographed during all seasons and it should contain species from every continent.</li>
</ul>
</p>

<h2>3. Data exploration</h2>

<h4>3.3 Features and it's characteristics</h4>